In [1]:
from datasets import load_from_disk, load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import numpy as np
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer, AutoModelForMaskedLM, BertConfig, BertModel, BertForPreTraining, BertForMaskedLM, AutoModel, PretrainedConfig, AutoConfig

import sys 
import os
sys.path.append(os.path.abspath(".."))

from collate import DataCollatorForLanguageModelingSpan


dataset = load_from_disk("../batchlarge")
dataset = dataset.remove_columns(["species_name", "__index_level_0__"])


tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")

# This way we don't load weights
# https://stackoverflow.com/questions/65072694/make-sure-bert-model-does-not-load-pretrained-weights
# TODO AutConfig or AutoModel? i guess it doesn't matter
config = PretrainedConfig.from_pretrained("togethercomputer/m2-bert-80M-2k")
model = BertForMaskedLM(config)

You are using a model of type m2_bert to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


In [2]:
%env WANDB_PROJECT=singlesamplednam2

data_collator = DataCollatorForLanguageModelingSpan(tokenizer, mlm=True, mlm_probability = 0.02, span_length = 6)

training_args = TrainingArguments(
    output_dir="./results",
    
    max_steps=1,
    
    seed=17,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=16,
    
    logging_strategy="steps",
    logging_steps=1,
    
    evaluation_strategy="no",
    
    #dataloader_num_workers=4,
    #dataloader_prefetch_factor=2,
    run_name="4batch256_monarchhf",
    report_to="none" #"wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator
)

trainer.train()

model.save_pretrained(".")

env: WANDB_PROJECT=singlesamplednam2


/opt/modules/i12g/anaconda/envs/m2-mixer/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,10.475000


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32
    })
})